In [1]:
from elasticsearch import Elasticsearch 
import util as ut
from elasticsearch import helpers
from rdflib import Graph
import collections

In [2]:
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])

In [91]:
# get all indexes
es.indices.get_alias("*")

{'.monitoring-es-7-2019.06.05': {'aliases': {}},
 '.monitoring-kibana-7-2019.06.05': {'aliases': {}},
 'rdf': {'aliases': {}},
 '.kibana_task_manager': {'aliases': {}},
 '.monitoring-kibana-7-2019.06.06': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}},
 '.monitoring-es-7-2019.06.06': {'aliases': {}}}

In [92]:
#if es.indices.exists(index='rdf'):
#es.indices.delete(index='opal_distribution')
#es.indices.delete(index='opal_dataset')

### Create index for dcat:Dataset

In [93]:
'''create_dcat_dataset_index_body = {
    "settings": {
        "number_of_shards": 1,# for distributed indexing.
        "number_of_replicas": 0},
    "mappings": {
            "properties": {
                "title": {"type": "text"},
                "description": {"type": "text"},
                "issued": {"type": "text"},#date
                "modified":{"type":"date"},
                "identifier":{"type":"keyword"}, 
                "keyword":{"type":"text"}, 
                "language":{"type":"keyword"},
                "accrualPeriodicity":{"type":"keyword"},
                "theme":{"type":"keyword"},
                "hasQualityMeasurement":{"type":"keyword"}
            }
    } 
}
create_dcat_distribution_index_body = {
    "settings": {
        "number_of_shards": 1,# for distributed indexing.
        "number_of_replicas": 0},
    "mappings": {
            "properties": {
                "title": {"type": "text"},
                "description": {"type": "text"},
                "issued": {"type": "text"},#date
                "issued": {"type": "date",
                          "format":"E, dd MMM YYY etc.."},#date
                
                "modified":{"type":"date"},
                "rights":{"type":"keyword"},
                "license":{"type":"keyword"},
                "accessURL":{"type":"keyword"},
                "mediaType":{"type":"keyword"},
                "byteSize":{"type":"integer"},
            }
    } 
}
es.indices.create(index='opal_dataset', body=create_dcat_dataset_index_body)
es.indices.create(index='opal_distribution', body=create_dcat_distribution_index_body)'''

'create_dcat_dataset_index_body = {\n    "settings": {\n        "number_of_shards": 1,# for distributed indexing.\n        "number_of_replicas": 0},\n    "mappings": {\n            "properties": {\n                "title": {"type": "text"},\n                "description": {"type": "text"},\n                "issued": {"type": "text"},#date\n                "modified":{"type":"date"},\n                "identifier":{"type":"keyword"}, \n                "keyword":{"type":"text"}, \n                "language":{"type":"keyword"},\n                "accrualPeriodicity":{"type":"keyword"},\n                "theme":{"type":"keyword"},\n                "hasQualityMeasurement":{"type":"keyword"}\n            }\n    } \n}\ncreate_dcat_distribution_index_body = {\n    "settings": {\n        "number_of_shards": 1,# for distributed indexing.\n        "number_of_replicas": 0},\n    "mappings": {\n            "properties": {\n                "title": {"type": "text"},\n                "description": {"t

In [94]:
es.indices.create(index='opal_dataset')
es.indices.create(index='opal_distribution')

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'opal_distribution'}

In [95]:
g = Graph()
g.parse("KG/10000.nt", format="n3")
#g.parse("/home/demir/Desktop/10000dataset.nt", format="n3")

<Graph identifier=N3bfbfebcd4e34f229a5d8df1c80cea75 (<class 'rdflib.graph.Graph'>)>

In [98]:
'{0} number of triples'.format(len(g))

'100001 number of triples'

In [99]:
prefix_uri='http://projekt-opal.de/'
#profiles=['dataset/','distribution/']
profiles=[prefix_uri+'dataset/',prefix_uri+'distribution/']

In [100]:
dataset_documents=dict()
distribution_documents=dict()

for t in g:
    s,p,o=str(t[0]),str(t[1]),str(t[2])
    
    if profiles[0] in s:
        # Important
        dataset_documents.setdefault(s, {}).setdefault(p,[]).append(o)
    elif profiles[1] in s:
        # Important
        distribution_documents.setdefault(s, {}).setdefault(p,[]).append(o)
    else:
        continue

In [101]:
for dataset_uri,body in dataset_documents.items():
    es.create(index='opal_dataset',body=body,id=dataset_uri)
    

In [102]:
for dataset_uri,body in distribution_documents.items():
    es.create(index='opal_distribution',body=body,id=dataset_uri)
    

In [103]:
def print_search_stats(results):
    print("=" * 80)
    print(
        "Total %d found in %dms" % (results["hits"]["total"]["value"], results["took"])
    )
    print("-" * 80)

### Example Queries 

In [107]:
# to get the number of documents
print(es.count(index='opal_dataset'))

print(es.count(index='opal_distribution'))
# gets all documents
#es.search(index="opal_dataset")

{'count': 1250, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
{'count': 2204, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [162]:
# Search for documents whose description contains framework
result=es.search(index='opal_dataset',
         body={
             "query":{
                 "bool":{
                     "must":{"match":{"http://purl.org/dc/terms/description":"@fr"}}
                 }
             }
         })

print_search_stats(result)

Total 70 found in 2ms
--------------------------------------------------------------------------------


In [161]:
list(result['hits']['hits'][0]['_source'].keys())

['http://www.w3.org/ns/dqv#hasQualityMeasurement',
 'http://purl.org/dc/terms/spatial',
 'http://www.w3.org/ns/dcat#keyword',
 'http://www.w3.org/ns/dcat#distribution',
 'http://purl.org/dc/terms/description',
 'http://www.w3.org/ns/dcat#theme',
 'http://purl.org/dc/terms/title',
 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type']

In [140]:
hits_docs=result['hits']['hits']

In [152]:
for i in hits_docs:
    
    index=i['_id']
    
    
  #  description=i['_source']['http://purl.org/dc/terms/description']
 #   print(index)
#    print(description)
    distribution=i['_source']['http://www.w3.org/ns/dcat#distribution'][0]
   
    resp = es.get(index='opal_distribution', id=distribution)
    print(resp)

    break

{'_index': 'opal_distribution', '_type': '_doc', '_id': 'http://projekt-opal.de/distribution/https___europeandataportal_eu_set_distribution_4f55bce7_405c_4ff7_83f7_1744c7bfb0b6', '_version': 1, '_seq_no': 1977, '_primary_term': 1, 'found': True, '_source': {'http://purl.org/dc/terms/description': [''], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': ['http://www.w3.org/ns/dcat#Distribution'], 'http://purl.org/dc/terms/title': ['view'], 'http://www.w3.org/ns/dcat#accessURL': ['http://ext-geodata.lansstyrelsen.se/arcgis/services/WMS/LST_wms_miljodata/MapServer/WMSserver?']}}


In [163]:
# Search for documents whose description contains framework
result=es.search(index='opal_dataset',
         body={
             "query":{
                 "bool":{
                     "must":{"match":{"http://purl.org/dc/terms/description":" @fr"}}
                 }
             }
         })

print_search_stats(result)

Total 70 found in 2ms
--------------------------------------------------------------------------------
